# Lecture 23: Deep Learning Models

## 1. Convolutional Neural Networks in PyTorch

https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html

1.1 Import packages

In [1]:
import torch
from torch import nn
import torch.optim as optim

1.2 CNN class

In [2]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        #input shape (3,256,256)
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, #input channels
                      out_channels=16, #n_filter
                     kernel_size=5, #filter size
                     stride=1, #filter step
                     padding=2 #same output size
                     ), #shape (16,256,256)
            nn.ReLU(), #fully-connected layer
            nn.MaxPool2d(kernel_size=2)) #max pooling layer. 2x2 sampling
        #output shape (16,128,128)
        
        self.conv2 = nn.Sequential(nn.Conv2d(16, 32, 5, 1, 2), 
                                  nn.ReLU(),
                                  nn.MaxPool2d(2))
        #output shape (32,64,64)
        
        self.out = nn.Linear(32*64*64,10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        #Flatten
        x = x.view(x.size(0), -1)
        output = self.out(x)
        return output

## 2. Recurrent Neural Networks in Pytorch

2.1 LSTM class

In [3]:
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=100, output_size=1, batch_size=8):
        super().__init__()
        self.batch_size = batch_size
        self.hidden_size = hidden_size
        self.num_layers = 1
        
        #First LSTM layer: bidirectional
        self.lstm1 = nn.LSTM(input_size, hidden_size, num_layers=self.num_layers,batch_first=True, bidirectional=True)
        #Second LSTM layer
        self.lstm2 = nn.LSTM(input_size, hidden_size, num_layers=self.num_layers,batch_first=True)

        self.linear = nn.Linear(hidden_size, output_size)
        
        #dropout layer
        self._dropout1 = nn.Dropout(p=0.3)
        self._dropout2 = nn.Dropout(p=0.2)


    def forward(self, input_seq, train=False):
        device = torch.device("cuda")
        h0 = torch.zeros(self.num_layers+1, self.batch_size, self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers+1, self.batch_size, self.hidden_size).to(device)
        h1 = torch.zeros(self.num_layers, self.batch_size, self.hidden_size).to(device)
        c1 = torch.zeros(self.num_layers, self.batch_size, self.hidden_size).to(device)
        
        lstm_out, hidden_cell = self.lstm2(input_seq, (h1, c1))
        unpacked, unpacked_len = torch.nn.utils.rnn.pad_packed_sequence(lstm_out)
        
        lstm_out = unpacked
        predictions = self.linear(lstm_out[-1, :, :])
        
        return predictions